In [1]:
# !pip install ultralytics
# !pip install roboflow
# !pip install --upgrade ultralytics

In [2]:
%load_ext autoreload
%autoreload

In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2

import os
from ultralytics import YOLO
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional.detection.iou import intersection_over_union

from PIL import Image, ImageOps

import multiprocessing as mp

import mlflow

from tqdm.auto import tqdm

from roboflow import Roboflow

/home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# if __name__ == "__main__":
#     mp.set_start_method("spawn", force=True)


# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
# torch.set_default_device("cuda")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
GENERATOR = (
    torch.Generator(device=DEVICE) if torch.cuda.is_available() else torch.Generator()
)
# BATCH_SIZE = 128
# use_mlflow = True
# mlflow.set_tracking_uri("http://localhost:5000")

# plt.rcParams["figure.dpi"] = 100
# plt.rcParams["xtick.labelsize"] = 12
# plt.rcParams["ytick.labelsize"] = 12

In [5]:
torch.cuda.is_available()

True

In [6]:
!nvidia-smi

Tue Sep 16 15:12:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        Off |   00000000:01:00.0  On |                  N/A |
|  0%   46C    P3             73W /  340W |    1458MiB /  10240MiB |     35%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
DEVICE

'cuda'

In [ ]:
# !file yolo12s.pt

yolo12s.pt: Zip archive data, at least v0.0 to extract, compression method=store


In [11]:
model = YOLO("yolo12s.pt")
model.to(DEVICE)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_runnin

In [12]:
rf = Roboflow(api_key="PN6wLUL75bQSI1gmZcAE")
project = rf.workspace("elbrus-soa0u").project("my-first-project-o78dm")
version = project.version(3)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-3 in yolov12:: 100%|██████████| 667/667 [00:00<00:00, 8146.06it/s]


In [16]:
torch_generator = torch.Generator(device=DEVICE)
results = model.train(
    data="My-First-Project-1/data.yaml",
    epochs=50,
    imgsz=600,
    batch=4,
    device=DEVICE,
    workers=0,
    half=True,
)

Ultralytics 8.3.200 🚀 Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 9864MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=My-First-Project-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=600, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train31, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

Traceback (most recent call last):
  File "/home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/.venv/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 366, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/.venv/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 464, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/.venv/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 1634, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/.venv/lib/python3.12/site-packages/mlflo

MLflow: logging run_id(ef47ef0965584b0ab4a8e02f4c8a8ecf) to /home/dmitry/elbrus/Phase2/ds-phase-2-git/runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri /home/dmitry/elbrus/Phase2/ds-phase-2-git/runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
WARNING ⚠️ MLflow: Failed to initialize: Changing param values is not allowed. Param with key='batch' was already logged with value='8' for run ID='ef47ef0965584b0ab4a8e02f4c8a8ecf'. Attempted logging new value '4'.
WARNING ⚠️ MLflow: Not tracking this run
Image sizes 608 train, 608 val
Using 0 dataloader workers
Logging results to /home/dmitry/elbrus/Phase2/ds-phase-2-git/runs/detect/train31
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/50      3.05G      1.474      3.458      1.924          6        608: 100% ━━━━━━━━━━━━ 85/85 10.0it/s 8.5s.1ss
                 Class     Images  Instances      Box(P          R      

In [ ]:
# !mlflow ui --backend-store-uri /home/dmitry/elbrus/Phase2/ds-phase-2-git/runs/mlflow

INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:     Started parent process [236126]
INFO:     Started server process [236131]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [236129]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [236128]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [236130]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:54158 - "GET / HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:54172 - "GET /static-files/manifest.json HTTP/1.1" 200 OK
Traceback (most recent call last):
  File "/home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/.venv/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 366, in search_experiments
    exp = self._get_experiment(exp_id, view_type

In [ ]:
print(results[0].metrics)

AttributeError: 'Results' object has no attribute 'metrics'. See valid attributes below.

    A class for storing and manipulating inference results.

    This class provides comprehensive functionality for handling inference results from various
    Ultralytics models, including detection, segmentation, classification, and pose estimation.
    It supports visualization, data export, and various coordinate transformations.

    Attributes:
        orig_img (np.ndarray): The original image as a numpy array.
        orig_shape (tuple[int, int]): Original image shape in (height, width) format.
        boxes (Boxes | None): Detected bounding boxes.
        masks (Masks | None): Segmentation masks.
        probs (Probs | None): Classification probabilities.
        keypoints (Keypoints | None): Detected keypoints.
        obb (OBB | None): Oriented bounding boxes.
        speed (dict): Dictionary containing inference speed information.
        names (dict): Dictionary mapping class indices 

In [20]:
# https://s273vla.storage.yandex.net/rdisk/10bd7306c07b5dd7391bd5f5d18346babe3718f3ec8fb3d0ffeb430f1fde2861/68c97ac6/fKqInKw3d7bLFOeFnMGnhHEHR5XeuQX1aRK_6nFRqUVGDh1d405L9YNYMbctceMn42ep8o-puuzKN4dab3xe7G6UbbXRlyLcY7jcFAaD5Q6r8npumZHI4midPdWhecNq?uid=1130000060687031&filename=2022-12-05%2021-55-00.JPG&disposition=inline&hash=&limit=0&content_type=image%2Fjpeg&owner_uid=1130000060687031&fsize=3641394&hid=df4a40fe301170ee40db3dae16cbe00b&media_type=image&tknv=v3&etag=cad47c981527aeca8cc9329858d9c2fb&ts=63eec559fbd80&s=5099d3e52f022dc5445689f5b28eac331f52a5141b8842048bc2612c026355ef&pb=U2FsdGVkX18oESUkB_xfKCP5DiEQ6kNRetYNL2mRZMhODWpHRqVW507M5j6TjCPBOz_rhB8kiBxy3rTu80tiyYzq4VsziZboEBWYndMgVr804Xq_PAf4baM1jG7vtKeu
# url = "https://s273vla.storage.yandex.net/rdisk/10bd7306c07b5dd7391bd5f5d18346babe3718f3ec8fb3d0ffeb430f1fde2861/68c97ac6/fKqInKw3d7bLFOeFnMGnhHEHR5XeuQX1aRK_6nFRqUVGDh1d405L9YNYMbctceMn42ep8o-puuzKN4dab3xe7G6UbbXRlyLcY7jcFAaD5Q6r8npumZHI4midPdWhecNq?uid=1130000060687031&filename=2022-12-05%2021-55-00.JPG&disposition=inline&hash=&limit=0&content_type=image%2Fjpeg&owner_uid=1130000060687031&fsize=3641394&hid=df4a40fe301170ee40db3dae16cbe00b&media_type=image&tknv=v3&etag=cad47c981527aeca8cc9329858d9c2fb&ts=63eec559fbd80&s=5099d3e52f022dc5445689f5b28eac331f52a5141b8842048bc2612c026355ef&pb=U2FsdGVkX18oESUkB_xfKCP5DiEQ6kNRetYNL2mRZMhODWpHRqVW507M5j6TjCPBOz_rhB8kiBxy3rTu80tiyYzq4VsziZboEBWYndMgVr804Xq_PAf4baM1jG7vtKeu"

results = model.predict(
    source="data/test_imgs/chat-pheromones.jpg", conf=0.1, save=True
)


image 1/1 /home/dmitry/elbrus/Phase2/ds-phase-2-git/week2/day2_yolo/data/test_imgs/chat-pheromones.jpg: 480x608 1 Other animals, 2 Random cats, 8.7ms
Speed: 8.6ms preprocess, 8.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 608)
Results saved to /home/dmitry/elbrus/Phase2/ds-phase-2-git/runs/detect/train314


In [21]:
# Process results list
# for result in results:
#     boxes = result.boxes  # bbox нужен для детекции
#     masks = result.masks  # Маска нужна для сегментации
#     keypoints = result.keypoints  # Для определения позы
#     probs = result.probs  # Для классификации найденного объекта
#     obb = result.obb  # Ориентированный bbox
#     # result.show()  # display to screen
img_path = results[0].plot()  # plot() возвращает numpy array с боксами
plt.figure(figsize=(10, 10))
plt.imshow(img_path.astype("uint8"))
plt.axis("off")
plt.show()
;

<Figure size 1000x1000 with 1 Axes>

''

In [ ]:
#!/bin/bash

!echo "=== Проверка версии glibc на системе ==="
!ldd --version | head -n1

!echo -e "\n=== Проверка версии snap core20 ==="
!snap info core20 | grep installed

!echo -e "\n=== Проверка версии snap shotwell ==="
!snap info shotwell | grep installed

!echo -e "\n=== Проверка библиотек, используемых snap-пакетом shotwell ==="
!snap run --shell shotwell -c "ldd /usr/bin/shotwell | grep libc"

!echo -e "\n=== Проверка символа __libc_pthread_init в libc ==="
!nm -D /usr/lib/x86_64-linux-gnu/libc.so.6 | grep __libc_pthread_init || echo "Символ не найден"


=== Проверка версии glibc на системе ===
ldd (Ubuntu GLIBC 2.39-0ubuntu8.5) 2.39
/usr/bin/ldd: line 41: printf: write error: Broken pipe
/usr/bin/ldd: line 43: printf: write error: Broken pipe

=== Проверка версии snap core20 ===
  Base snaps are installed automatically when a snap package requires them.
  Only one of each type of base snap is ever installed.
installed:          20250526            (2599) 66MB base

=== Проверка версии snap shotwell ===

=== Проверка библиотек, используемых snap-пакетом shotwell ===
error: snap "shotwell" is not installed

=== Проверка символа __libc_pthread_init в libc ===
Символ не найден
